In [29]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pip install --upgrade xlrd

     |████████████████████████████████| 96 kB 2.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


# Lab 3 Assignment
- In the exercises, you will still use the same dataset after One-Hot Encoding
- The dependent variable is "default payment next month"

__Name:__ Priyanka Manoj Shewale

## Starting point - Import and Split dataset
- Split the dataset into training and testing set, with a ratio of 80:20

In [30]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_recall_curve

In [31]:
df = pd.read_excel('http://fengmai.net//download/courses/2020S-BIA652NT-Pub/Week10-Classification%20and%20Hyperparameter%20Search%20using%20Python/default%20of%20credit%20card%20clients.xls', header = 1).drop('ID', axis = 1)

col = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
df2 = pd.get_dummies(df, columns = col, drop_first=True)
df2.head()

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,...,PAY_5_8,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3,PAY_6_4,PAY_6_5,PAY_6_6,PAY_6_7,PAY_6_8
0,20000,24,3913,3102,689,0,0,0,0,689,...,0,0,0,0,0,0,0,0,0,0
1,120000,26,2682,1725,2682,3272,3455,3261,0,1000,...,0,0,0,1,0,0,0,0,0,0
2,90000,34,29239,14027,13559,14331,14948,15549,1518,1500,...,0,0,1,0,0,0,0,0,0,0
3,50000,37,46990,48233,49291,28314,28959,29547,2000,2019,...,0,0,1,0,0,0,0,0,0,0
4,50000,57,8617,5670,35835,20940,19146,19131,2000,36681,...,0,0,1,0,0,0,0,0,0,0


In [32]:
from sklearn.model_selection import train_test_split
y = df2['default payment next month'].values.astype(float)
x = df2.drop('default payment next month', axis = 1).values.astype(float)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

## Q1 - Grid-search both C and number of PCs 

Next, we use the gride search procedure to search for both the optimal C (inverse of L2 regularization parameter) and the number of principle components using cross validation on the train set.

- Define a parameter grid such that pca__n_components is 10 to 30, and add C as a parameter that can take the following values: [1000, 100, 10, 1, 0.1, 0.01]. Hint: use `pipe.get_params().keys()` to get the right key for C. 
- Which C and PC combination is the best?

In [33]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [34]:
##### YOUR CODE HERE ######
# build a pipeline which executes three steps in a row
pipeline = Pipeline(
     steps=[
    ('standardization', preprocessing.StandardScaler()),
    ('pca', PCA()),    
    ('lg', LogisticRegression(C=0.01, solver = 'lbfgs', max_iter = 1000))
])
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardization', 'pca', 'lg', 'standardization__copy', 'standardization__with_mean', 'standardization__with_std', 'pca__copy', 'pca__iterated_power', 'pca__n_components', 'pca__random_state', 'pca__svd_solver', 'pca__tol', 'pca__whiten', 'lg__C', 'lg__class_weight', 'lg__dual', 'lg__fit_intercept', 'lg__intercept_scaling', 'lg__l1_ratio', 'lg__max_iter', 'lg__multi_class', 'lg__n_jobs', 'lg__penalty', 'lg__random_state', 'lg__solver', 'lg__tol', 'lg__verbose', 'lg__warm_start'])

In [35]:
param_grid = {'pca__n_components': range(10, 31),'lg__C':[1000,100,10,1,0.1,0.01]}
grid = GridSearchCV(pipeline, cv = 3, param_grid = param_grid, scoring = 'roc_auc', refit=True, verbose=1)
grid_fit = grid.fit(x_train, y_train)

Fitting 3 folds for each of 126 candidates, totalling 378 fits


In [36]:
grid.best_estimator_

Pipeline(steps=[('standardization', StandardScaler()),
                ('pca', PCA(n_components=30)),
                ('lg', LogisticRegression(C=1, max_iter=1000))])

In [37]:
report(grid.cv_results_)

Model with rank: 1
Mean validation score: 0.764 (std: 0.006)
Parameters: {'lg__C': 1, 'pca__n_components': 30}

Model with rank: 2
Mean validation score: 0.764 (std: 0.006)
Parameters: {'lg__C': 10, 'pca__n_components': 30}

Model with rank: 3
Mean validation score: 0.764 (std: 0.006)
Parameters: {'lg__C': 0.1, 'pca__n_components': 29}



NOte :  The best C & PC value is C =1000 & PC_n_components = 30

## Q2 - Provide AUC in the test set
- Based on Q1, use the best chosen model to provide AUC for the test set

In [38]:
#### YOUR CODE HERE ####
y_test

array([0., 0., 0., ..., 1., 0., 0.])

In [39]:
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)

In [40]:
pca = PCA(n_components = 30)
pca.fit(x_train_s)

PCA(n_components=30)

In [41]:
x_train_pca = pca.transform(x_train_s)
x_test_pca = pca.transform(x_test_s)

In [42]:
mod1 = LogisticRegression(C=1000, solver = 'lbfgs', max_iter = 1000)
mod1.fit(x_train_pca, y_train)

LogisticRegression(C=1000, max_iter=1000)

In [43]:
y_predict_test = lg.predict(x_test_pca)
#y_predict_test = lg.predict(x_test_s)

In [44]:
#predict_prob = lg.predict_proba(x_test_s)
predict_prob = lg.predict_proba(x_test_pca)
predict_prob

array([[0.38914686, 0.61085314],
       [0.77571315, 0.22428685],
       [0.84969475, 0.15030525],
       ...,
       [0.8767248 , 0.1232752 ],
       [0.15808756, 0.84191244],
       [0.96406932, 0.03593068]])

In [45]:
fpr, tpr, thresholds = roc_curve(y_test, predict_prob[:, 1])
print('AUC: {:.2f}'.format(auc(fpr, tpr)))

AUC: 0.76


AUC is 0.76 with the best chosen model parameters

## Q3 - Confusion matrix, precision, recall, F-1
- When decision threshold is 0.4:
    - Provide Confusion matrix, precision, recall, F-1 for the test set

In [46]:
from sklearn.metrics import confusion_matrix, classification_report
print('classifier accuracy is {:.2f}'.format(lg.score(x_test_pca, y_test)))

cm = confusion_matrix(y_test, y_pred=y_predict_test)
print('confusion matrix is :\n', cm)

classifier accuracy is 0.81
confusion matrix is :
 [[4523  180]
 [ 937  360]]


In [47]:
print('classification report:\n', classification_report(y_test, y_predict_test))

classification report:
               precision    recall  f1-score   support

         0.0       0.83      0.96      0.89      4703
         1.0       0.67      0.28      0.39      1297

    accuracy                           0.81      6000
   macro avg       0.75      0.62      0.64      6000
weighted avg       0.79      0.81      0.78      6000



## Q4 - Optimal decision threshold

If the average cost of a false positive prediction is 1, and the cost of a false negative prediction is 2.5, what is the optimal decision threshold that can minimize cost in the test set given your predicted_prob? Provide answer up to 2 decimal places. 

In [48]:
dt_cost = []
out =[]
for i in np.arange(0.0, 1.0, 0.01):
    print(f"decision threshold: {i:.2f}")
    #### YOUR CODE HERE ####
    out.append(i)
    predict_prob_given=(predict_prob[:,1]>= i).astype(int)
    cm_new=confusion_matrix(y_test,predict_prob_given)
    fn_new=cm_new[1][0]
    fp_new=cm_new[0][1]
    scores=2*fn_new+fp_new
    dt_cost.append(scores)
min_pos=dt_cost.index(min(dt_cost))   
min_pos

decision threshold: 0.00
decision threshold: 0.01
decision threshold: 0.02
decision threshold: 0.03
decision threshold: 0.04
decision threshold: 0.05
decision threshold: 0.06
decision threshold: 0.07
decision threshold: 0.08
decision threshold: 0.09
decision threshold: 0.10
decision threshold: 0.11
decision threshold: 0.12
decision threshold: 0.13
decision threshold: 0.14
decision threshold: 0.15
decision threshold: 0.16
decision threshold: 0.17
decision threshold: 0.18
decision threshold: 0.19
decision threshold: 0.20
decision threshold: 0.21
decision threshold: 0.22
decision threshold: 0.23
decision threshold: 0.24
decision threshold: 0.25
decision threshold: 0.26
decision threshold: 0.27
decision threshold: 0.28
decision threshold: 0.29
decision threshold: 0.30
decision threshold: 0.31
decision threshold: 0.32
decision threshold: 0.33
decision threshold: 0.34
decision threshold: 0.35
decision threshold: 0.36
decision threshold: 0.37
decision threshold: 0.38
decision threshold: 0.39


29

In [49]:
out[min_pos]

0.29

In [50]:
dt_cost[min_pos]

1802

Note : The optimal decision threshold is 0.28 with the cost as 1803

In [51]:
tn, fp, fn, tp = cm.ravel()
print(f'tn: {tn}, fp: {fp}, fn: {fn}, tp: {tp}')

tn: 4523, fp: 180, fn: 937, tp: 360
